In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import logging
import os
from pathlib import Path
import io
import json
import zipfile
import requests

In [ ]:
def get_qualtrics_survey(use_labels=True):
    """Export Qualtrics Survey Responses via API Call According to the Documentation.

    Input
    :param survey_id: Qualtrics ID for Survey to export.
    """
    surveyId = #"SV_8iwwlvdP2dqQWZU" Update this field
    fileFormat = "csv"
    dataCenter = "nielsenscarborough.iad1"
    apiToken = "gZRjDAnNzFDDO7kuV8HDXI9GPG2xMg1rUhDOsu5I"

    # Setting static parameters
    requestCheckProgress = 0.0
    progressStatus = "inProgress"
    baseUrl = "https://{0}.qualtrics.com/API/v3/surveys/{1}/export-responses/".format(
        dataCenter, surveyId
    )
    headers = {
        "content-type": "application/json",
        "x-api-token": apiToken,
    }

    # Step 1: Creating Data Export
    downloadRequestUrl = baseUrl
    downloadRequestPayload = {"format": fileFormat, "useLabels": use_labels}

    downloadRequestPayload = json.dumps(downloadRequestPayload)

    downloadRequestResponse = requests.request(
        "POST", downloadRequestUrl, data=downloadRequestPayload, headers=headers
    )
    progressId = downloadRequestResponse.json()["result"]["progressId"]
    print(downloadRequestResponse.text)

    # Step 2: Checking on Data Export Progress and waiting until export is ready
    while progressStatus != "complete" and progressStatus != "failed":
        print("progressStatus = " + progressStatus)
        requestCheckUrl = baseUrl + progressId
        requestCheckResponse = requests.request("GET", requestCheckUrl, headers=headers)
        requestCheckProgress = requestCheckResponse.json()["result"]["percentComplete"]
        print("Download is " + str(requestCheckProgress) + " complete")
        progressStatus = requestCheckResponse.json()["result"]["status"]

    # step 2.1: Check for error
    if progressStatus == "failed":
        raise Exception("export failed")

    fileId = requestCheckResponse.json()["result"]["fileId"]

    # Step 3: Downloading file
    requestDownloadUrl = baseUrl + fileId + "/file"
    requestDownload = requests.request(
        "GET", requestDownloadUrl, headers=headers, stream=True
    )

    # Step 4: Unzipping the file and read as pandas dataframe
    print("Reading zip to pandas")
    r_zip = zipfile.ZipFile(io.BytesIO(requestDownload.content))
    df = pd.read_csv(r_zip.open(r_zip.namelist()[0]), sep=",", dtype=str)
    print("Successfully read from API into Pandas dataframe")
    return df

In [ ]:
df.drop([0,1], axis=0, inplace=True)

In [ ]:
df['overall_exp'].value_counts()

In [ ]:
df['materials_login'].value_counts()